# Importing necessary libraries
import os
import logging
from fastapi import FastAPI, HTTPException
from pytrends.request import TrendReq
import pandas as pd

# Creating a FastAPI instance
app = FastAPI()

# List of countries for Google Trends data
COUNTRY_LIST = ['AT', 'BE', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'GR', 'ES',
                'FI', 'FR', 'HU', 'IE', 'IS', 'IT', 'LU', 'ME',
                'MT', 'NL', 'NO', 'PL', 'PT', 'SE', 'SI', 'GB',
                'LT', 'LV']

# Function to fetch Google Trends data
def fetch_google_trends_data(category, countries=COUNTRY_LIST, start_date='2014-01-01', end_date='2024-04-07'):
    """
    Fetch Google Trends data for the specified category and countries.
    """

# Function to clean DataFrame
def clean_dataframe(df):
    """
    Clean the DataFrame by dropping NaN values and duplicates.
    """

# Endpoint to clean Google Trends data
@app.get("/clean-google-trends-data/{category}")
def clean_google_trends_data(category: str):
    """
    Clean Google Trends data for the specified category and save the cleaned data to a CSV file.
    """

# Endpoint to save CSV data
@app.get("/save-csv/{category}")
def save_csv_handler(category: str):
    """
    Save the cleaned CSV data to a specified directory.
    """


In [5]:
import os
import logging
from fastapi import FastAPI, HTTPException
from pytrends.request import TrendReq
import pandas as pd

app = FastAPI()

COUNTRY_LIST = ['AT', 'BE', 'CH', 'CY', 'CZ', 'DE', 'DK', 'EE', 'GR', 'ES',
                'FI', 'FR', 'HU', 'IE', 'IS', 'IT', 'LU', 'ME',
                'MT', 'NL', 'NO', 'PL', 'PT', 'SE', 'SI', 'GB',
                'LT', 'LV']

def fetch_google_trends_data(category, countries=COUNTRY_LIST, start_date='2014-01-01', end_date='2024-04-07'):
    pytrends = TrendReq(hl='es', tz=360)
    trends_data = {}
    for country in countries:
        try:
            pytrends.build_payload(kw_list=[category], timeframe=f'{start_date} {end_date}', geo=country)
            data = pytrends.interest_over_time()
            if data.empty:
                raise ValueError(f"No data available for the category and country '{country}'.")
            weekly_data = data.resample('M').mean()
            trends_data[country] = weekly_data.to_dict()
        except Exception as e:
            logging.error(f"Error fetching Google Trends data for the country '{country}': {e}")
            raise HTTPException(status_code=500, detail=f"Error fetching Google Trends data for the country '{country}'")

    return trends_data

category = "Unemployment benefits"
df = fetch_google_trends_data(category)
df

def clean_dataframe(df):
    cleaned_df= df.dropna().drop_duplicates()
    return cleaned_df
    

@app.get("/clean-google-trends-data/{category}")
def clean_google_trends_data(category: str):
    try:
        # Fetch Google Trends data
        trends_data = fetch_google_trends_data(category)

        # Convert to DataFrame
        df = pd.DataFrame.from_dict(trends_data)

        # Clean DataFrame
        cleaned_df = clean_dataframe(df)

        # Save cleaned data to CSV
        cleaned_df.to_csv(f"{category}_cleaned_data.csv", index=True)

        return {"message": f"Data cleaned and saved successfully"}
    except Exception as e:
        logging.error(f"Error cleaning Google Trends data for category '{category}': {e}")
        raise HTTPException(status_code=500, detail=f"Error cleaning Google Trends data for category '{category}'")

@app.get("/save-csv/{category}")
def save_csv_handler(category: str):
    try:
        # Load cleaned data from CSV
        cleaned_df = pd.read_csv(f"{category}_cleaned_data.csv")

        # Create directory if it doesn't exist
        directory = 'data'
        if not os.path.exists(directory):
            os.makedirs(directory)

        # Save DataFrame as CSV
        cleaned_df.to_csv(os.path.join(directory, f'data_{category}.csv'), index=True, encoding='utf-8')
        
        return {"message": f"CSV file for '{category}' saved successfully"}
    
    
    except Exception as e:
        logging.error(f"Error saving CSV file for topic '{category}': {e}")
        raise HTTPException(status_code=500, detail=f"Error saving CSV file for category '{category}'")


ERROR:root:Error fetching Google Trends data for the country 'BE': The request failed: Google returned a response with code 429


HTTPException: 500: Error fetching Google Trends data for the country 'BE'